# Statistical Analysis
This notebook loads the dataset, calculates statistics, and selects target users.

Eyad Medhat 221100279
/ Hady Aly 221101190
/ Mohamed Mahfouz 221101743
/ Omar Mady 221100745

In [1]:
from utils import *

df = load_and_sample_data()
if df is not None:
    print(f"Loaded DataFrame with shape: {df.shape}")
    print(df.head())
else:
    print("Failed to load data.")

 Found cached sample at: ..\data\ml-20m\ratings_cleaned_sampled.csv
Loaded DataFrame with shape: (1000000, 3)
   userId  movieId  rating
0   49400     4011       5
1  118685      786       3
2   80510     1247       3
3  124963     1721       1
4   31031     1982       4


In [2]:
current_users = df['userId'].nunique()
current_items = df['movieId'].nunique()
current_interactions = len(df)

print(f"Current Users: {current_users}")
print(f"Current Items: {current_items}")
print(f"Current Interactions (Ratings): {current_interactions}")

Current Users: 96345
Current Items: 1000
Current Interactions (Ratings): 1000000


In [ ]:
user_stats_df = Mean(df, group_col='userId', target_col='rating')


print("Number of ratings per user (rating_count_per_user) calculated.")
print(user_stats_df.head())

save_csv(user_stats_df,'stats_per_user')

Starting manual loop for Group: 'userId' -> Target: 'rating'...


In [ ]:
item_stats_df = Mean(df, group_col='movieId', target_col='rating')


print("Number of ratings per item (stats_per_item) calculated.")
print(item_stats_df.head())

save_csv(item_stats_df,'stats_per_item')

# Item Plotting

In [ ]:
# Ascendingly order the total number of ratings per item
sorted_counts = item_stats_df['rating_count_per_movie'].sort_values(ascending=True).values

# Plot the distribution per item
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(sorted_counts, color='blue', linewidth=1.5)

# Logarithmic scale to visualize the long-tail distribution
ax.set_yscale('log')

ax.set_title('Distribution of Ratings per Item (Log Scale)')
ax.set_xlabel('Items (Sorted by Popularity)')
ax.set_ylabel('Number of Ratings (Log Scale)')
ax.grid(True, which="both", linestyle='--', alpha=0.5)

save_plot(fig, 'item_rating_distribution_ascending')
plt.show()


In [ ]:
# 1. Sort the DataFrame by the correct column name
sorted_counts = item_stats_df.sort_values(by='rating_count_per_movie', ascending=True)

# Histogram: Frequency of Rating Counts
fig = plt.figure(figsize=(12, 6))

# 2. Plot using the SORTED DataFrame and CORRECT column name
plt.hist(sorted_counts['rating_count_per_movie'], bins=50, edgecolor='black', color='blue', log=True)

plt.title('Distribution of Rating Counts (Histogram)')
plt.xlabel('Number of Ratings')
plt.ylabel('Number of Items (Frequency)')

save_plot(fig, 'item_rating_distribution_ascending_hist')
plt.show()

# Item Grouping

In [ ]:
# 1. Sort items by Average Rating (Manual-friendly approach)
# We already have item_stats_df from Section 1.6
item_stats_sorted = item_stats_df.sort_values(by='mean_rating_per_movie', ascending=True).reset_index(drop=True)

total_items = len(item_stats_sorted)

# 2. Define the index boundaries based on percentages
# Req: <1%, 1-5%, 5-10%, 10-20%, 20-30%, 30-40%, 40-50%, 50-60%, 60-70%, 70-100%
cuts = [0, 0.01, 0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 1.0]
group_labels = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10']

# 3. Manually assign groups based on row position (index)
item_groups = []

# Loop through the sorted dataframe
for index, row in item_stats_sorted.iterrows():
    # Calculate relative position (0.0 to 1.0)
    position = (index + 1) / total_items
    
    # Determine group
    assigned_group = ""
    for i in range(len(cuts) - 1):
        if position <= cuts[i+1]:
            assigned_group = group_labels[i]
            break
            
    item_groups.append(assigned_group)

item_stats_sorted['group'] = item_groups

print("Point 1.8 Complete: Items grouped by percentile position manually.")
print(item_stats_sorted[['movieId', 'mean_rating_per_movie', 'group']].head())
print(item_stats_sorted[['movieId', 'mean_rating_per_movie', 'group']].tail())

save_csv(item_stats_sorted, 'grouped_items.csv')


In [ ]:
# Initialize dictionary
group_sums = {label: 0 for label in group_labels}

# Manual Loop to sum ratings per group
for index, row in item_stats_sorted.iterrows():
    g = row['group']
    n_ratings = row['rating_count_per_movie']
    group_sums[g] += n_ratings

# Convert to DataFrame for easier printing/plotting
group_counts_df = pd.DataFrame(list(group_sums.items()), columns=['group', 'total_ratings'])

# Order ascendingly as required
group_counts_sorted = group_counts_df.sort_values(by='total_ratings', ascending=True)

print("Total ratings per group")
print(group_counts_sorted)


save_csv(group_counts_sorted,'movie_count_per_group.csv')

In [ ]:
# --- Plot 1: Unsorted Groups ---
fig = plt.figure(figsize=(12, 6))
group_counts_df.plot(kind='bar', color='skyblue', edgecolor='black', log=True, ax=plt.gca())

plt.title('Distribution of Total Ratings per Group (Log Scale)')
plt.xlabel('Group (G1-G10)')
plt.ylabel('Total Number of Ratings (Log Scale)')

# Force x-axis labels to match the group names
# rot=0 makes them horizontal for readability
plt.xticks(range(len(group_counts_df)), group_counts_df['group'], rotation=0) 

plt.grid(axis='y', linestyle='--', alpha=0.5, which='both')
save_plot(fig, 'group_counts_hist_not_sorted')
plt.show()


# --- Plot 2: Sorted Groups ---
fig = plt.figure(figsize=(12, 6))
group_counts_sorted.plot(kind='bar', color='skyblue', edgecolor='black', log=True, ax=plt.gca())

plt.title('Distribution of Total Ratings per Group (Sorted, Log Scale)')
plt.xlabel('Group (G1-G10)')
plt.ylabel('Total Number of Ratings (Log Scale)')

# Force x-axis labels to match the group names of the SORTED data
plt.xticks(range(len(group_counts_sorted)), group_counts_sorted['group'], rotation=0)

plt.grid(axis='y', linestyle='--', alpha=0.5, which='both')
save_plot(fig, 'group_counts_hist_sorted')
plt.show()

In [ ]:

# --- 4. User Selection ---
if item_stats_df is not None and user_stats_df is not None:
    
    # Recalculate percentages if needed (or ensure they exist)
    # We recalculate to be safe as loaded CSV might be old format
    total_items_count = len(item_stats_df)
    user_stats_df['pct_rated'] = (user_stats_df['rating_count_per_user'] / total_items_count) * 100
    
    print(f"Total Items: {total_items_count}")
    print("------------------------------------------------")

    selected_data = []

    # Random Selection Logic
    # U1: < 2%
    u1_id = get_random_user_and_check(
        df=user_stats_df,
        storage_list=selected_data,
        condition_mask=(user_stats_df['pct_rated'] < 2.0),
        label="U1 (Sparse)",
        condition_text="< 2%",
        seed=42
    )

    # U2: 2% - 5%
    u2_id = get_random_user_and_check(
        df=user_stats_df,
        storage_list=selected_data,
        condition_mask=(user_stats_df['pct_rated'] >= 2.0) & (user_stats_df['pct_rated'] < 5.0),
        label="U2 (Medium)",
        condition_text="2% - 5%",
        seed=42
    )

    # U3: 5% - 10%
    u3_id = get_random_user_and_check(
        df=user_stats_df,
        storage_list=selected_data,
        condition_mask=(user_stats_df['pct_rated'] >= 5.0) & (user_stats_df['pct_rated'] < 10.0),
        label="U3 (Dense)",
        condition_text="5% - 10%",
        seed=42
    )

    print("\nFinal Selected IDs for Random Targets:")
    print(f"U1: {u1_id}, U2: {u2_id}, U3: {u3_id}")

    # Save Selected Users
    target_users_df = pd.DataFrame(selected_data)
    save_csv(target_users_df, 'Selected_Target_Users.csv')
else:
    print("Skipping selection: Missing required stats.")

In [ ]:
# 1. Get the DataFrame of the lowest rated items
# NOTE: change 'avg_rating' to your actual rating column name if different
lowest_rated_df = (
    item_stats_df
    .sort_values(by='mean_rating_per_movie', ascending=True)
    .head(2)
    .reset_index(drop=True)
)

if lowest_rated_df is not None and len(lowest_rated_df) >= 2:
    # 2. Extract IDs for further use (if needed)
    I1 = lowest_rated_df.iloc[0]['movieId']
    I2 = lowest_rated_df.iloc[1]['movieId']
    print(f"Target Item 1: {I1}")
    print(f"Target Item 2: {I2}")

    # 3. Save to CSV
    # This will save to E:\grad_year_sem1\irs_project\SECTION1_DimensionalityReduction\results\tables\lowest_two_rateditems.csv
    save_csv(lowest_rated_df, 'lowest_two_rateditems')

In [ ]:
# Ensure main dataframe is loaded
if 'df' not in locals() or df is None:
    print("Main dataframe 'df' not found. Loading...")
    # Adjust path if needed for notebook location
    import os
    if os.path.exists(os.path.join('..', 'data', 'ml-20m', 'ratings_cleaned.csv')):
         df = load_data(os.path.join('..', 'data', 'ml-20m', 'ratings_cleaned.csv'))
    else:
         df = load_data(os.path.join('ml-20m', 'ratings_cleaned.csv'))

# Load Target Users
try:
    users_csv_path = r'../results/tables/Selected_Target_Users.csv'
    users_df = pd.read_csv(users_csv_path)
    target_user_ids = users_df['UserId'].tolist()
    print(f"Loaded {len(target_user_ids)} Target Users: {target_user_ids}")
except Exception as e:
    print(f"Error loading target users: {e}")
    target_user_ids = []

# Load Target Items
try:
    items_csv_path = r'../results/tables/lowest_two_rateditems.csv'
    items_df = pd.read_csv(items_csv_path)
    target_item_ids = items_df['movieId'].tolist()
    print(f"Loaded {len(target_item_ids)} Target Items: {target_item_ids}")
except Exception as e:
    print(f"Error loading target items: {e}")
    target_item_ids = []

# --- Calculation Functions ---

# Optimized Execution
if df is not None:
    print("Grouping data by User and Item for fast lookup...")
    # Map: UserId -> Set of MovieIds
    user_to_movies = df.groupby('userId')['movieId'].apply(set)
    # Map: MovieId -> Set of UserIds
    movie_to_users = df.groupby('movieId')['userId'].apply(set)
    
    # 1. Target Users Analysis
    for uid in target_user_ids:
        if uid not in user_to_movies:
            print(f"User {uid} not found in ratings.")
            continue
            
        target_movies = user_to_movies[uid]
        results = []
        
        for other_uid, other_movies in user_to_movies.items():
            if other_uid == uid: continue
            
            # Overlap
            overlap = len(target_movies.intersection(other_movies))
            if overlap > 0:
                results.append({'target_user': uid, 'other_user': other_uid, 'No_common_users': overlap})
        
        # Save
        res_df = pd.DataFrame(results).sort_values(by='No_common_users', ascending=False)
        save_csv(res_df, f'Target_User_{uid}_Overlap')
        print(f"Saved overlap stats for User {uid} ({len(res_df)} neighbors)")

    # 2. Target Items Analysis
    for iid in target_item_ids:
        if iid not in movie_to_users:
            print(f"Item {iid} not found in ratings.")
            continue
            
        target_users = movie_to_users[iid]
        results = []
        
        for other_iid, other_users in movie_to_users.items():
            if other_iid == iid: continue
            
            # Overlap
            overlap = len(target_users.intersection(other_users))
            if overlap > 0:
                results.append({'target_item': iid, 'other_item': other_iid, 'No_coRated_items': overlap})
                
        # Save
        res_df = pd.DataFrame(results).sort_values(by='No_coRated_items', ascending=False)
        save_csv(res_df, f'Target_Item_{iid}_Overlap')
        print(f"Saved overlap stats for Item {iid} ({len(res_df)} related items)")

In [ ]:
def main():
    print("--- Starting Cold User Extraction ---")
    
    # 1. Define Paths and Constants
    # stats_per_user.csv is in results/tables
    input_path = os.path.join('..', 'results', 'tables', 'stats_per_user.csv')
    
    TOTAL_ITEMS = 26744  # Constant from project metadata
    THRESHOLD_PCT = 2.0  # Cold user threshold <= 2%

    if not os.path.exists(input_path):
        print(f"Error: Could not find input file: {input_path}")
        return

    # 2. Load Data
    print("Loading user stats...")
    df = pd.read_csv(input_path)
    print(f"  Total Users Loaded: {len(df)}")

    # 3. Calculate Percentage
    # Formula: (Count / Total_Items) * 100
    df['pct_rated'] = (df['rating_count_per_user'] / TOTAL_ITEMS) * 100

    # 4. Filter Cold Users
    print(f"Filtering for users with <= {THRESHOLD_PCT}% ratings ({int(TOTAL_ITEMS * THRESHOLD_PCT / 100)} items)...")
    cold_users_df = df[df['pct_rated'] <= THRESHOLD_PCT].copy()
    
    count = len(cold_users_df)
    print(f"  Cold Users Found: {count} ({count/len(df)*100:.2f}% of total)")

    # 5. Save Results
    # We want to save to results/tables/cold_user.csv
    # utils.save_csv logic: it saves to results/tables/filename.csv relative to where it thinks 'results' is.
    # We'll just use save_csv from utils. It handles the 'results' folder discovery.
    # Note: 'cold_user' requested filename.
    
    save_csv(cold_users_df, 'cold_user')
    
    print("--- Extraction Complete ---")

if __name__ == "__main__":
    main()


In [ ]:
def main():
    print("--- Starting Medium User Extraction ---")
    
    # 1. Define Paths and Constants
    input_path = os.path.join('..', 'results', 'tables', 'stats_per_user.csv')
    
    TOTAL_ITEMS = 26744
    MIN_PCT = 2.0
    MAX_PCT = 5.0

    if not os.path.exists(input_path):
        print(f"Error: Could not find input file: {input_path}")
        return

    # 2. Load Data
    print("Loading user stats...")
    df = pd.read_csv(input_path)

    # 3. Calculate Percentage
    df['pct_rated'] = (df['rating_count_per_user'] / TOTAL_ITEMS) * 100

    # 4. Filter Medium Users
    print(f"Filtering for users with {MIN_PCT}% <= ratings <= {MAX_PCT}%...")
    
    medium_users_df = df[
        (df['pct_rated'] >= MIN_PCT) & 
        (df['pct_rated'] <= MAX_PCT)
    ].copy()
    
    count = len(medium_users_df)
    print(f"  Medium Users Found: {count} ({count/len(df)*100:.2f}% of total)")

    # 5. Save Results
    save_csv(medium_users_df, 'medium_user')
    
    print("--- Extraction Complete ---")

if __name__ == "__main__":
    main()


In [ ]:
def main():
    print("--- Starting Rich User Extraction ---")
    
    # 1. Define Paths and Constants
    input_path = os.path.join('..', 'results', 'tables', 'stats_per_user.csv')
    
    TOTAL_ITEMS = 26744
    THRESHOLD_PCT = 10.0  # Rich user threshold > 10%

    if not os.path.exists(input_path):
        print(f"Error: Could not find input file: {input_path}")
        return

    # 2. Load Data
    print("Loading user stats...")
    df = pd.read_csv(input_path)

    # 3. Calculate Percentage
    df['pct_rated'] = (df['rating_count_per_user'] / TOTAL_ITEMS) * 100

    # 4. Filter Rich Users
    print(f"Filtering for users with > {THRESHOLD_PCT}% ratings...")
    
    rich_users_df = df[df['pct_rated'] > THRESHOLD_PCT].copy()
    
    count = len(rich_users_df)
    print(f"  Rich Users Found: {count} ({count/len(df)*100:.2f}% of total)")

    # 5. Save Results
    save_csv(rich_users_df, 'rich_user')
    
    print("--- Extraction Complete ---")

if __name__ == "__main__":
    main()


In [ ]:
def main():
    print("--- Starting Item Popularity Extraction ---")
    
    # 1. Define Paths and Constants
    input_path = os.path.join('..', 'results', 'tables', 'stats_per_item.csv')
    
    TOTAL_USERS = 138493  # Constant from project metadata
    
    if not os.path.exists(input_path):
        print(f"Error: Could not find input file: {input_path}")
        return

    # 2. Load Data
    print("Loading item stats...")
    df = pd.read_csv(input_path)
    total_items = len(df)

    # 3. Calculate Percentage
    # Formula: (Rating_Count / Total_Users) * 100
    df['pct_rated'] = (df['rating_count_per_movie'] / TOTAL_USERS) * 100

    # 4. Categorize Items
    print("Categorizing items...")
    
    # Low Popularity: <= 2%
    low_pop_df = df[df['pct_rated'] <= 2.0].copy()
    print(f"  Low Popularity Items (<= 2%): {len(low_pop_df)} ({len(low_pop_df)/total_items*100:.2f}%)")
    
    # Medium Popularity: 2% < x <= 5%
    med_pop_df = df[
        (df['pct_rated'] > 2.0) & 
        (df['pct_rated'] <= 5.0)
    ].copy()
    print(f"  Medium Popularity Items (2% < x <= 5%): {len(med_pop_df)} ({len(med_pop_df)/total_items*100:.2f}%)")
    
    # High Popularity: > 5%
    high_pop_df = df[df['pct_rated'] > 5.0].copy()
    print(f"  High Popularity Items (> 5%): {len(high_pop_df)} ({len(high_pop_df)/total_items*100:.2f}%)")

    # 5. Save Results
    save_csv(low_pop_df, 'low_popularity_items')
    save_csv(med_pop_df, 'medium_popularity_items')
    save_csv(high_pop_df, 'high_popularity_items')
    
    print("--- Extraction Complete ---")

if __name__ == "__main__":
    main()
